
<p>
<font size="9" face="Arial">
<h1 align="center"> Ship-Identification-and-Tracking-via-Satellite-Imagery
 </h1>
</font>
</p>


--------------

<p>
<font size="4" face="Arial">In this notebook, I will create a model that predicts weather there is ship in the image or not.</font>
</p>

<p>
<font size="6" face="Arial">
<h2 align="left"> 1. Dataset Information </h2> 
</font>
</p>
<p>
<font size="5" face="Arial">
<h2 align="left"> Context </h2>
</font>
</p>

    
<p>
<font size="4" face="Arial">
Satellite imagery provides unique insights into various markets, including agriculture, defense and intelligence, energy, and finance. New commercial imagery providers, such as Planet, are using constellations of small satellites to capture images of the entire Earth every day.

The aim of this dataset is to help address the difficult task of detecting the location of large ships in satellite images. Automating this process can be applied to many issues including monitoring port activity levels and supply chain analysis.
</font>
</p>
</br>
<p>
<font size="5" face="Arial">
<h2 align="left"> Content </h2>
</font>
<p>
<font size="4" face="Arial">
The dataset consists of image chips extracted from Planet satellite imagery collected over the San Francisco Bay and San Pedro Bay areas of California. It includes 4000 80x80 RGB images labeled with either a "ship" or "no-ship" classification. Image chips were derived from PlanetScope full-frame visual scene products, which are orthorectified to a 3 meter pixel size.
</font>
</p>
</br>   




<p>
<font size="4" face="Arial">    
The dataset is also distributed as a JSON formatted text file shipsnet.json. The loaded object contains data, label, scene_ids, and location lists.
<ol type="square">
<li>**label**: Valued 1 or 0, representing the "ship" class and "no-ship" class, respectively.</li>
<li>**scene id**: The unique identifier of the PlanetScope visual scene the image chip was extracted from. The scene id can be used with the Planet API to discover and download the entire scene.</li>
<li>**longitude_latitude**: The longitude and latitude coordinates of the image center point, with values separated by a single underscore.</li>    
</ol>    
The pixel value data for each 80x80 RGB image is stored as a list of 19200 integers within the data list. The first 6400 entries contain the red channel values, the next 6400 the green, and the final 6400 the blue. The image is stored in row-major order, so that the first 80 entries of the array are the red channel values of the first row of the image.
</font>
</p>
<p>
<font size="5" face="Arial">
<h2 align="left"> Class Labels </h2>
</font>
</p>

<p>
<font size="4" face="Arial">
The "ship" class includes 1000 images. Images in this class are near-centered on the body of a single ship. Ships of different sizes, orientations, and atmospheric collection conditions are included. Example images from this class are shown below.
</font>  
</p>

<figure class="half" style="center">
    <img style="width:700px" src="https://i.imgur.com/tLsSoTz.png">
</figure>
<p>
<font size="4" face="Arial">
The "no-ship" class includes 3000 images. A third of these are a random sampling of different landcover features - water, vegetion, bare earth, buildings, etc. - that do not include any portion of an ship. The next third are "partial ships" that contain only a portion of an ship, but not enough to meet the full definition of the "ship" class. The last third are images that have previously been mislabeled by machine learning models, typically caused by bright pixels or strong linear features. Example images from this class are shown below.
</font>
</p>
<figure class="half" style="center">
    <img style="width:700px" src="https://i.imgur.com/Q3daQMC.png">
</figure>


<p>
<font size="6" face="Arial">
<h2 align="left"> 2. Loading and Checking Dataset </h2> 
</font>
</p>
   



<p>
<font size="5" face="Arial">
<h2 align="left"> Loading and Reshaping </h2>
</font>
</p>

In [ ]:
import numpy as np
from numpy import expand_dims
import pandas as pd
import json
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import keras
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

In [ ]:
with open('../input/ships-in-satellite-imagery/shipsnet.json') as data_file:
    dataset = json.load(data_file)
shipsnet= pd.DataFrame(dataset)
shipsnet.head()

In [ ]:
shipsnet.info()

<p>
<font size="4" face="Arial">
<li> We need just two columns in this project which are data and labels. </li>
</font>
</p>

In [ ]:
shipsnet = shipsnet[["data", "labels"]]
shipsnet.head()

In [ ]:
len(shipsnet["data"].iloc[0])

 <p>
<font size="4" face="Arial">
<li> As you can see, the pixel value data for each 80x80 RGB image is stored as a list of 19200 integers within the data list. The first 6400 entries contain the red channel values, the next 6400 the green, and the final 6400 the blue. </li>
</font>
</p>

In [ ]:
ship_images = shipsnet["labels"].value_counts()[0]
no_ship_images = shipsnet["labels"].value_counts()[1]
print("Number of the ship_images :{}".format(ship_images),"\n")
print("Number of the ship_images :{}".format(no_ship_images))

In [ ]:
# Turning the json information into numpy array and then assign it as x and y variables
x = np.array(dataset['data']).astype('uint8')
y = np.array(dataset['labels']).astype('uint8')

In [ ]:
x.shape

<p>
<font size="4" face="Arial">
<li> The current data for each image is one row of flattened 19200 data points representing the RGB values of each pixel. So we need to reshape it. After reshaping, each item in new x variable will be 3 lists. Each of these lists will be RGB values for each pixel for the length and width of the image. </li>
</font>
</p>

In [ ]:
x_reshaped = x.reshape([-1, 3, 80, 80])

x_reshaped.shape

<p>
<font size="4" face="Arial">
<li> We need to change the order of the dimensions to get the correct format to plot the images. The last value of the shape of x_reshaped will be 3, this number represent the channel. (RGB) </li>
</font>
</p>    

<img src="https://e2eml.school/images/image_processing/three_d_array.png" class="center" style="width:400px">



In [ ]:
x_reshaped = x.reshape([-1, 3, 80, 80]).transpose([0,2,3,1])
x_reshaped.shape

In [ ]:
y.shape

<p>
<font size="4" face="Arial">
<li> y variable consist of label values, 1 or 0. We need to convert them binary class matrix. </li>
</font>
</p>

In [ ]:
y_reshaped = to_categorical(y, num_classes=2)

y_reshaped.shape

In [ ]:
y_reshaped

<p>
<font size="5" face="Arial">
<h2 align="left"> Exploring the Images </h2>
</font>
</p>

<p>
<font size="4" face="Arial">
<li> A quick look at images with and without ships. </li>
</font>
</p>

In [ ]:
image_no_ship = x_reshaped[y==0]
image_ship = x_reshaped[y==1]

def plot(a,b):
    
    plt.figure(figsize=(15, 15))
    for i, k in enumerate(range(1,9)):
        if i < 4:
            plt.subplot(2,4,k)
            plt.title('Not A Ship')
            plt.imshow(image_no_ship[i+2])
            plt.axis("off")
        else:
            plt.subplot(2,4,k)
            plt.title('Ship')
            plt.imshow(image_ship[i+15])
            plt.axis("off")
            
    plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0.25)

#Implementation of the function 

plot(image_no_ship, image_ship)

<p>
<font size="4" face="Arial">
<li> A quick look at pixel intensity of images. </li>
</font>
</p>

In [ ]:
def plotHistogram(ship, not_ship):

    plt.figure(figsize = (10,7))
    plt.subplot(2,2,1)
    plt.imshow(ship)
    plt.axis('off')
    plt.title('Ship')
    histo = plt.subplot(2,2,2)
    histo.set_ylabel('Count', fontweight = "bold")
    histo.set_xlabel('Pixel Intensity', fontweight = "bold")
    n_bins = 30
    plt.hist(ship[:,:,0].flatten(), bins = n_bins, lw = 0, color = 'r', alpha = 0.5);
    plt.hist(ship[:,:,1].flatten(), bins = n_bins, lw = 0, color = 'g', alpha = 0.5);
    plt.hist(ship[:,:,2].flatten(), bins = n_bins, lw = 0, color = 'b', alpha = 0.5);
    plt.show()
    print("Minimum pixel value of this image: {}".format(ship.min()))
    print("Maximum pixel value of this image: {}".format(ship.max()))
    plt.figure(figsize = (10,7))
    plt.subplot(2,2,3)
    plt.imshow(not_ship)
    plt.axis('off')
    plt.title('Not A Ship')
    histo = plt.subplot(2,2,4)
    histo.set_ylabel('Count', fontweight = "bold")
    histo.set_xlabel('Pixel Intensity', fontweight = "bold")
    n_bins = 30
    plt.hist(not_ship[:,:,0].flatten(), bins = n_bins, lw = 0, color = 'r', alpha = 0.5);
    plt.hist(not_ship[:,:,1].flatten(), bins = n_bins, lw = 0, color = 'g', alpha = 0.5);
    plt.hist(not_ship[:,:,2].flatten(), bins = n_bins, lw = 0, color = 'b', alpha = 0.5);
    plt.show()
    print("Minimum pixel value of this image: {}".format(not_ship.min()))
    print("Maximum pixel value of this image: {}".format(not_ship.max()))
#Implementation of the function

for i in range (20,23):
    plotHistogram(x_reshaped[y==1][i], x_reshaped[y==0][i])

<p>
<font size="4" face="Arial">
<li> Now let’s take a quick view of each channels in the two image. </li>
</font>
</p>

In [ ]:
my_list = [(0, 'R channel'), (1, 'G channel'), (2, 'B channel')]

plt.figure(figsize = (15,15))

for i, k in my_list:
    plt.subplot(1,3,i+1)
    plt.title(k)
    plt.ylabel('Height {}'.format(x_reshaped[y==0][5].shape[0]))
    plt.xlabel('Width {}'.format(x_reshaped[y==0][5].shape[1]))
    plt.imshow(x_reshaped[y==0][5][ : , : , i])

In [ ]:
my_list = [(0, 'R channel'), (1, 'G channel'), (2, 'B channel')]

plt.figure(figsize = (15,15))

for i, k in my_list:
    plt.subplot(1,3,i+1)
    plt.title(k)
    plt.ylabel('Height {}'.format(x_reshaped[y==0][5].shape[0]))
    plt.xlabel('Width {}'.format(x_reshaped[y==0][5].shape[1]))
    plt.imshow(x_reshaped[y==1][5][ : , : , i])

<p>
<font size="6" face="Arial">
<h2 align="left"> 3. Modelling </h2> 
</font>
</p>


<p>
<font size="5" face="Arial">
<h2 align="left"> Preparing of Train and Test Data </h2>
</font>
</p>

<p>
<font size="4" face="Arial">
<li> First step is normalizing x data. </li>
</font>
</p>

In [ ]:
x_reshaped = x_reshaped / 255

In [ ]:
x_reshaped[0][0][0] # Normalized RGB values of the firs pixel of the first image in the dataset.

In [ ]:
n_bins = 30
plt.hist(x_reshaped[y == 0][0][:,:,0].flatten(), bins = n_bins, lw = 0, color = 'r', alpha = 0.5);
plt.hist(x_reshaped[y == 0][0][:,:,1].flatten(), bins = n_bins, lw = 0, color = 'g', alpha = 0.5);
plt.hist(x_reshaped[y == 0][0][:,:,2].flatten(), bins = n_bins, lw = 0, color = 'b', alpha = 0.5);
plt.ylabel('Count', fontweight = "bold")
plt.xlabel('Pixel Intensity', fontweight = "bold")
plt.title("Histogram of normalized data")
plt.show()

<p>
<font size="4" face="Arial">
<li> Data is ready for splitting as train and test. </li>
</font>
</p>

In [ ]:
x_train_1, x_test, y_train_1, y_test = train_test_split(x_reshaped, y_reshaped,
                                                        test_size = 0.20, random_state = 42)


x_train, x_val, y_train, y_val = train_test_split(x_train_1, y_train_1, 
                                                  test_size = 0.25, random_state = 42)


print("x_train shape",x_train.shape)
print("x_test shape",x_test.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)
print("y_train shape",x_val.shape)
print("y_test shape",y_val.shape)

In [ ]:
x_train.shape

<p>
<font size="5" face="Arial">
<h2 align="left"> Implementation of Artificial Neural Network (ANN) </h2>
</font>
</p>

In [ ]:
from keras import callbacks
model = Sequential()
model.add(Flatten(input_shape=[80, 80, 3]))
model.add(Dense(200, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

history = model.fit(x_train, y_train, epochs = 100, validation_data=(x_val, y_val), callbacks = [earlystopping])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
pd.DataFrame(history.history).plot();

<p>
<font size="4" face="Arial">
<li> Now I will add two more layer and change activation function as softmax . </li>
</font>
</p>

In [ ]:
from keras import callbacks
model = Sequential()
model.add(Flatten(input_shape=[80, 80, 3]))
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

history = model.fit(x_train, y_train, epochs = 100, validation_data=(x_val, y_val), callbacks = [earlystopping])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
pd.DataFrame(history.history).plot();

<p>
<font size="4" face="Arial">
<li> Now we can predict our test data with ANN. I will check the first image of the test data. </li>
</font>
</p>

In [ ]:
prediction = model.predict(x_test)
pd.Series(prediction[0], index=["Not A Ship", "Ship"])

In [ ]:
plt.imshow(x_test[0])
plt.axis("off")
plt.show()

<p>
<font size="4" face="Arial">
<li> Let's take a look at the images that are estimated to be ship, although not a ship. </li>
</font>
</p>

In [ ]:
predicted_data = pd.DataFrame(prediction, columns=["Not A Ship", "Ship"])
predicted_data.head(3)

In [ ]:
y_test_data = pd.DataFrame(y_test, columns=["Not A Ship", "Ship"])
y_test_data.head(3)

In [ ]:
predicted_data['There is a Ship'] = y_test[:, 1]
predicted_data.head()

In [ ]:
predicted_data["Difference"] = predicted_data["Ship"] - predicted_data["There is a Ship"]
predicted_data

<p>
<font size="4" face="Arial">
<li> If difference is high, that means the image predicted as Ship, altought not a ship. To see such predicted images, we need to sort the difference column from largest to smallest. </li>
</font>
</p>

In [ ]:
predicted_data.sort_values('Difference', ascending=False).head(10)

<p>
<font size="4" face="Arial">
<li> Now, I want to show some of these images and their pixel intensity histograms. </li>
</font>
</p>

In [ ]:
indexes = predicted_data.sort_values('Difference', ascending = False).head(4).index.to_list()

def plotHistogram(image_index):

    plt.figure(figsize = (10,7))
    plt.subplot(2,2,1)
    plt.imshow(x_test[image_index])
    plt.axis('off')
    plt.title('There is no ship. But predicted as a ship.')
    histo = plt.subplot(2,2,2)
    histo.set_ylabel('Count', fontweight = "bold")
    histo.set_xlabel('Pixel Intensity', fontweight = "bold")
    n_bins = 30
    plt.hist(x_test[image_index][:,:,0].flatten(), bins = n_bins, lw = 0, color = 'r', alpha = 0.5);
    plt.hist(x_test[image_index][:,:,1].flatten(), bins = n_bins, lw = 0, color = 'g', alpha = 0.5);
    plt.hist(x_test[image_index][:,:,2].flatten(), bins = n_bins, lw = 0, color = 'b', alpha = 0.5);
    plt.show()


#Implementation of the function

for i in indexes:
    plotHistogram(i)

<p>
<font size="4" face="Arial">
<li> Now, let's take a look at the images that are estimated to not a ship, although there is a ship. </li>
</font>
</p>

In [ ]:
predicted_data.sort_values('Difference', ascending=True).head(10)

<p>
<font size="4" face="Arial">
<li> Now, I want to show some of these images and their pixel intensity histograms. </li>
</font>
</p>


In [ ]:
indexes = predicted_data.sort_values('Difference', ascending = True).head(4).index.to_list()

def plotHistogram(image_index):

    plt.figure(figsize = (10,7))
    plt.subplot(2,2,1)
    plt.imshow(x_test[image_index])
    plt.axis('off')
    plt.title('There is a ship. But predicted as not a ship.')
    histo = plt.subplot(2,2,2)
    histo.set_ylabel('Count', fontweight = "bold")
    histo.set_xlabel('Pixel Intensity', fontweight = "bold")
    n_bins = 30
    plt.hist(x_test[image_index][:,:,0].flatten(), bins = n_bins, lw = 0, color = 'r', alpha = 0.5);
    plt.hist(x_test[image_index][:,:,1].flatten(), bins = n_bins, lw = 0, color = 'g', alpha = 0.5);
    plt.hist(x_test[image_index][:,:,2].flatten(), bins = n_bins, lw = 0, color = 'b', alpha = 0.5);
    plt.show()


#Implementation of the function

for i in indexes:
    plotHistogram(i)

<p>
<font size="5" face="Arial">
<h2 align="left"> Implementation of Convolutional Neural Network (CNN) </h2>
</font>
</p>

In [ ]:
from keras import callbacks
model = Sequential()
#
model.add(Conv2D(filters = 64, kernel_size = (4,4),padding = 'Same', 
                 activation ='relu', input_shape = (80,80,3)))
model.add(MaxPool2D(pool_size=(5,5)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(3,3), strides=(1,1)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 16, kernel_size = (2,2),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(3,3), strides=(1,1)))
model.add(Dropout(0.25))

# Fully connected
model.add(Flatten())
model.add(Dense(200, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)
history = model.fit(x_train, y_train, epochs = 100, validation_data=(x_val, y_val), callbacks = [earlystopping])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
pd.DataFrame(history.history).plot();

<p>
<font size="4" face="Arial">
<li> Data Augmentation. </li>
</font>
</p>

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=5,  
        zoom_range = 0.1,
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False, 
        vertical_flip=False)  

datagen.fit(x_train)

<p>
<font size="4" face="Arial">
<li> Data augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data. Data augmentation techniques such as cropping, padding, and horizontal flipping are commonly used to train large neural networks. </li>
<li> Let's see how it works. I will apply little data augmentation to an image.  </li>
</font>
</p>

In [ ]:
data = x_reshaped[y==1][15]
# expand dimension to one sample
samples = expand_dims(data, 0)
# create image data augmentation generator
datag = ImageDataGenerator(brightness_range=[0.2,1.0],
                          zoom_range=[0.5,1.0],
                          horizontal_flip=True,
                          rotation_range=90)
# prepare iterator
it = datag.flow(samples, batch_size=1)
# generate samples and plot
plt.figure(figsize = (10,10))
for i in range(9):
    # define subplot
    plt.subplot(3,3,i+1)
    # generate batch of images
    batch = it.next()
    # convert to unsigned integers for viewing
    image = batch[0].astype('uint8')
    # plot raw pixel data
    plt.imshow(image)
    # show the figure
plt.show()

<p>
<font size="4" face="Arial">
<li> Now I will apply data augmentation to train data and fit again.  </li>
</font>
</p>

In [ ]:
history = model.fit(datagen.flow(x_train, y_train), epochs = 100, 
                    validation_data=(x_val, y_val), callbacks = [earlystopping])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
from sklearn import metrics
import seaborn as sns
Y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# Compute the confusion matrix

print("\n""Test Accuracy Score : ",metrics.accuracy_score(Y_true, Y_pred_classes),"\n")

fig, axis = plt.subplots(1, 3, figsize=(20,6))
axis[0].plot(history.history['val_accuracy'], label='val_acc')
axis[0].set_title("Validation Accuracy")
axis[0].set_xlabel("Epochs")
axis[0].set_ylabel("Val. Acc.")
axis[1].plot(history.history['accuracy'], label='acc')
axis[1].set_title("Training Accuracy")
axis[1].set_xlabel("Epochs")
axis[0].set_ylabel("Train. Acc.")
axis[2].plot(history.history['val_loss'], label='val_loss')
axis[2].set_title("Test Loss")
axis[2].set_xlabel("Epochs")
axis[2].set_ylabel("Loss")

plt.show()

confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# Plot the confusion matrix
f,ax = plt.subplots(figsize=(7, 7))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()

plt.show()

In [ ]:
pd.DataFrame(history.history).plot();

<p>
<font size="4" face="Arial">
<li> Now we can predict our test data with CNN. I check the first image of the test data.  </li>
</font>
</p>

In [ ]:
prediction = model.predict(x_test)
pd.Series(prediction[0], index=["Not A Ship", "Ship"])

<p>
<font size="4" face="Arial">
<li> You can see that we get a result with a much higher probability than the model we created with ANN. </li>
<li> Let's take a look at the images that are estimated to be ship, although not a ship. </li>
</font>
</p>

In [ ]:
predicted_data = pd.DataFrame(prediction, columns=["Not A Ship", "Ship"])
predicted_data.head(3)

In [ ]:
y_test_data = pd.DataFrame(y_test, columns=["Not A Ship", "Ship"])
y_test_data.head(3)

In [ ]:
predicted_data['There is a Ship'] = y_test[:, 1]
predicted_data.head()

In [ ]:
predicted_data["Difference"] = predicted_data["Ship"] - predicted_data["There is a Ship"]
predicted_data

<p>
<font size="4" face="Arial">
<li> If difference is high, that means the image predicted as Ship, altought not a ship. To see such predicted images, we need to sort the difference column from largest to smallest. </li>
</font>
</p>

In [ ]:
predicted_data.sort_values('Difference', ascending=False).head(10)

<p>
<font size="4" face="Arial">
<li> Now, I want to show some of these images and their pixel intensity histograms. </li>
</font>
</p>

In [ ]:
indexes = predicted_data.sort_values('Difference', ascending = False).head(4).index.to_list()

def plotHistogram(image_index):

    plt.figure(figsize = (10,7))
    plt.subplot(2,2,1)
    plt.imshow(x_test[image_index])
    plt.axis('off')
    plt.title('There is no ship. But predicted as a ship.')
    histo = plt.subplot(2,2,2)
    histo.set_ylabel('Count', fontweight = "bold")
    histo.set_xlabel('Pixel Intensity', fontweight = "bold")
    n_bins = 30
    plt.hist(x_test[image_index][:,:,0].flatten(), bins = n_bins, lw = 0, color = 'r', alpha = 0.5);
    plt.hist(x_test[image_index][:,:,1].flatten(), bins = n_bins, lw = 0, color = 'g', alpha = 0.5);
    plt.hist(x_test[image_index][:,:,2].flatten(), bins = n_bins, lw = 0, color = 'b', alpha = 0.5);
    plt.show()


#Implementation of the function

for i in indexes:
    plotHistogram(i)

<p>
<font size="4" face="Arial">
<li> In conclusion, we achieve the best test accuracy with Convolutional Neurol Network.</li>
</font>
</p>